In [1]:
import collectionBooks

ROOT = '/Users/andrewlevin/Library/CloudStorage/OneDrive-СУНЦМГУШколаим.А.Н.Колмогорова,МГУим.М.В.Ломоносова/1 🏛 Biblioteka'
STORE = '../data/store'

collection = collectionBooks.Collection(root=ROOT, store=STORE)


# Tree Changer

In [ ]:
ROOT = 'data/biblioteka-sample'

# ROOT = 'Simple'

import json
from path_tree_generator import PathTree
import utils
import pathlib

pt = PathTree(
    ROOT,
    read_stat=True,
    relative_paths=False,
    paths_as_posix=True,
)
print(pt.human_readable())

data = json.loads(pt.json())

#utils.write_json(data, pathlib.Path('raw-tree.json'))

In [ ]:
import pathlib
import hashlib

          
def walker(data):
    
    data = element_processing(data)
    
    if data['entry_type'] == 'dir':
        print('\n')
        print('Walker Dir: ', data['name'])
        
        
        remove_list = []
        for idx, child in enumerate(data['children']):
            print('\t', 'Child:', child['name'])
            if child.get('name') == '.DS_Store':
                print('\t', 'Remove DS')
                remove_list.append(idx)
        
        for idx in remove_list[::-1]:
            print('idx', idx)
            del data['children'][idx]
    
        
        for idx, child in enumerate(data['children']):
            data['children'][idx] = walker(data['children'][idx])
                
                
    return data


def element_processing(data):
    
    data['stat']['md5'] = ''
    
    if data['entry_type'] == 'dir':
        pass
        
    elif data['entry_type'] == 'file':
        
        md5 = hashlib.md5(pathlib.Path(data['path']).read_bytes()).hexdigest()
        data['stat']['md5'] = md5
    
    return data




raw_data = read_json('raw-tree.json')

print(raw_data)

walker_data = walker(raw_data)

write_json(walker_data, 'tree.json')


In [ ]:
dictionary = {
   'Novel': 'Pride and Prejudice',
   'year': '1813',
   'author': 'Jane Austen',
   'character': 'Elizabeth Bennet'
}

# Iterating over all the keys of a dictionary and printing them one by one
for key in dictionary:
    print(key)
    print()


    

# Tree Generator


In [ ]:
import json

def list_of_paths2treeV2(paths=None, datas: list = []):
    if paths is None:
        paths = []

    def attach(tree, branch, data, parent = []):
        if not branch:
            return

        parts = branch.split('/', 1)
        name = parts[0]
        
        
        print('Parent')
        print(parent)
        parent.append(name)
        
        
        print('path_with_parent: ')
        path_with_parent = '/'.join(parent)
        path_with_parent = pathlib.Path(path_with_parent)
        print(path_with_parent)
        print("Tree:")
        print(json.dumps(tree, indent=4, ensure_ascii=False))
        print()
        print(parts)
        print("Name: ", name)
        idVshort = utils.get_very_short_id(pathlib.Path(branch))
        print(idVshort)
        print()
        print()
        
        # name = data['id_very_short']
        if len(parts) == 1:
            tree['children'][name] = {'type': 'FILE'}
            if data:
                for key in data.keys():
                    tree['children'][name][key] = data[key]

        else:
            childs = parts[1]
            if not tree['children'].get(name):
                tree['children'][name] = {
                    'type': 'DIR',
                    'children': {}
                }

            attach(tree['children'][name], childs, data, parent)

    tree_inp = {'children': {}, 'type': 'DIR'}
    for idx, path in enumerate(paths):
        attach(tree_inp, path, datas[idx])

    return tree_inp

In [ ]:
import treeBooks 
import utils
import pathlib


def generate_json_tree_of_collection(collection: collectionBooks.Collection):
    book_list = []
    book_data = []
    for identifier, book in collection.books.items():
        
        tree_data = {}
    
        tree_data['cover'] = book.data.get('cover')
        tree_data['id_very_short'] = utils.get_very_short_id(pathlib.Path(book.relative))
        #tree_data['md5_relative'] = book.md5_of_relative
        #tree_data['identifier'] = collectionBooks.get_identifier(book.relative)
        #tree_data['title'] = book.path.name
        
        book_list.append(book.relative.as_posix())
        book_data.append(tree_data)
        
        
    tree_books = list_of_paths2treeV2(book_list, book_data)
    
    treeBooks.print_directory_tree(tree_books)
    
    treeBooks.print_directory_tree_as_json(tree_books)

    
    
    tree_path = pathlib.Path(STORE).joinpath('tree2.json')
    
    utils.write_json(tree_path, tree_books)

    
    

generate_json_tree_of_collection(collection)

In [ ]:
import treeBooks 
import utils
import pathlib


def create_elements_json(collection: collectionBooks.Collection):
    scroll_books = {}
    for identifier, book in collection.books.items():
        
        book_elem = {}
        book_elem['cover'] = book.data.get('cover')
        book_elem['md5_relative'] = 'book.md5_of_relative'
        book_elem['identifier'] = 'identifier'
        book_elem['title'] = book.path.name
        book_elem['relative'] = book.relative.as_posix()
        if book.data.get('cover_ratio'):
            book_elem['cover_ratio'] = book.data.get('cover_ratio')
        
        scroll_books[book.data.get('id_short')] = book_elem
        
        
    scroll_data = {
        'udate': utils.get_utime(),
        'elements': scroll_books
    }
    
    scroll_path = pathlib.Path(STORE).joinpath('jsons/elements.json')
    
    utils.write_json(scroll_path, scroll_data)

    
    

create_elements_json(collection)

In [ ]:
import tree

if True:
    paths_input = [
        'dir-0/file',
        'dir-A/dir1/',
        'dir-A/dir1/file11',
        'dir-A/dir1/file10',
        'dir-A/dir2/file21',
        'dir-A/dir2/file22',
        'dir-A/file3',
        'dir-B/alpha/beta/gamma/delta',
        'dir-B/alpha/beta/gamma/delta/',
        'dir-C/file4',
        'dir-C/file5'
    ]

    got_tree = tree.list_of_paths2tree(paths_input)

    tree.print_directory_tree(got_tree)
    print('---' * 10)

    tree.print_directory_tree_as_json(got_tree)
    print('---' * 10)

    got_paths = tree.tree2list_of_paths(got_tree)
    print('\n'.join(got_paths))

In [ ]:
import utils
import json

from path_tree_generator import PathTree

ROOT  = '../data/biblioteka-sample'

tree = PathTree(ROOT).dict()

tree = utils.normalize_tree(tree)

tree = utils.remove_DS_Store_files(tree)

tree = utils.sort_tree(tree)


tree = utils.remove_some_keys(tree)

tree = utils.modify_some_keys(tree)


print(json.dumps(tree, indent=4, ensure_ascii=False))


In [ ]:
tree = utils.modify_some_keys(tree)


print(json.dumps(tree, indent=4, ensure_ascii=False))

In [ ]:
import pathlib

def sandbox(tree: dict = {}):
    path = tree.get('path')
    id = utils.get_very_short_id(pathlib.Path(path))
    print(path)
    print(id)
    print()
    
    tree['id'] = id

    if not tree.get('children'):
        return tree
    
    for child_tree in tree['children']:
        sandbox(child_tree)

    return tree


sandbox(tree)


In [ ]:
# Make Scroll


scroll = {
    'elements': {
    }
}



SCROLL_ELEMENT_DEFAULT = {
    'name': 'NAME',
    'id_long': 'ID_LONG',
    'id_short': 'ID_SHORT',
    'path': 'PATH',
    'type': 'TYPE',
    'ext': 'EXT'
}


TREE_NODE_DEFAULT = {
    'name': 'NAME',
    'id_short': 'ID_SHORT',
    'id_long': 'ID_LONG',
    'path': 'PATH',
    'type': 'TYPE',
    'childre': None,
}


def make_scroll(tree: dict = {}):
    path = tree.get('path')
    id = utils.get_very_short_id(pathlib.Path(path))
    print(path)
    print(id)
    print()
    
    
    element = {
        'name': tree.get('name'),
        'path': tree.get('path'),
        'type': tree.get('type'),
    }
    
    scroll['elements'][id] = element

    if not tree.get('children'):
        return tree
    
    for child_tree in tree['children']:
        make_scroll(child_tree)

    return tree


make_scroll(tree)

print(scroll)


In [ ]:
import pathlib
import utils

def generate_diffrent_id(tree: dict = {}):
    path = tree.get('path')
    id_short = utils.get_id_short(pathlib.Path(path))
    id_long = utils.get_id_long(pathlib.Path(path))
    tree['id_short'] = id_short
    tree['id_long'] = id_long
    

    if not tree.get('children'):
        return tree
    
    for child_tree in tree['children']:
        generate_diffrent_id(child_tree)

    return tree


sandbox(tree)


# Collection With Tree

In [ ]:
import collectionTools


collectionTools.export_all_jsons(collection)


In [ ]:
import utils
import pathlib

def export_tree_json2(collection: collectionBooks.Collection):

    tree2 = collection.tree.copy()
    name, children = remake_tree(tree2)
    
    tree_json = {
        'udate': utils.get_datetime(),
        'tree': {
            name: children
        }
    }

    output_tree_path = collection.store.path.joinpath('jsons/tree-2.json')
    utils.write_json(output_tree_path, tree_json)

    
def remake_tree(tree: dict = {}):
    id_short = utils.get_id_short(pathlib.Path(tree.get('path')))
    print(id_short)
    children = {}
    
    if not tree.get('children'):
        children = None
    
    else:
        for idx, child_tree in enumerate(tree['children']):
            ch_name, ch_children = remake_tree(child_tree)
            children[ch_name] = ch_children

    return id_short, children


    
export_tree_json2(collection)